In [37]:
import json
import os, sys
from sqlalchemy import create_engine
import pandas as pd 
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [39]:
dir = os.path.dirname
path1 = dir(dir(os.getcwd()))
sys.path.append(path1)

In [40]:
df = pd.read_csv("../../data/fire_archive_M6_96619.csv")


In [41]:
df = df[[ "latitude", "longitude", "scan", "track", "acq_time", "frp", "type" ]]

In [42]:
df

,latitude,longitude,scan,track,acq_time,frp,type
0,-11.8070,142.0583,1.0,1.0,56,6.6,0
1,-11.7924,142.0850,1.0,1.0,56,11.3,0
2,-12.8398,132.8744,3.1,1.7,57,23.1,0
3,-14.4306,143.3035,1.1,1.1,57,6.5,0
4,-12.4953,131.4897,4.0,1.9,57,27.6,0
...,...,...,...,...,...,...,...
36006,-28.0656,131.6334,1.2,1.1,1644,14.2,0
36007,-28.2625,128.5494,1.0,1.0,1644,6.4,0
36008,-28.2941,128.5123,1.0,1.0,1644,5.5,0
36009,-28.3001,128.5530,1.0,1.0,1644,5.7,0


In [43]:
X = df.drop("type", axis=1)
y = df["type"]

In [44]:
#from utils_.sql_functions import load_sql

In [45]:
def gridSearch(X, y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

    reg_log = Pipeline(steps=[
        ('pca', PCA(n_components=0.9, random_state=42)),
        ('classifier', SVC())])

    logistic_params = {'classifier': [LogisticRegression()]}
    random_forest_params = {'classifier': [KNeighborsClassifier()]}
    svm_params = {'classifier': [SVC()]}

    search_space = [
        logistic_params,
        random_forest_params,
        svm_params]

    clf = GridSearchCV(estimator = reg_log,
                    param_grid = search_space,
                    cv = 2,
                    verbose=1,
                    n_jobs=-1)

    return clf 

querry = """
        SELECT fire_id, latitude, longitude, scan, track, acq_date, acq_time, frp, fire_type
        FROM australia_fires.fire_archive_M6_96619
        UNION ALL
        SELECT fire_id, latitude, longitude, scan, track, acq_date, acq_time, frp, fire_type 
        FROM australia_fires.fire_archive_V1_96617
        """
with open(dir(path1) + os.sep + 'config' + os.sep + 'bd_info.json') as json_file:
        db_conf = json.load(json_file)

engine = create_engine('mysql+pymysql://' + db_conf['USER'] + ':' + db_conf['PASSWORD'] + '@' + db_conf['IP_DNS'] + ':' + str(db_conf['PORT'])+ '/' + db_conf['BD_NAME'])
con = engine.connect()
df = pd.read_sql_query(querry, con=con)
df

In [46]:
clf = gridSearch(X, y) 
clf.fit(X, y)
clf.best_estimator_
clf.best_score_

Fitting 2 folds for each of 3 candidates, totalling 6 fits


0.9904195947766863